# GRPO로 LLM 미세 조정

이 노트북은 `trl` 라이브러리를 사용하여 GRPO로 LLM을 미세 조정하는 방법을 보여줍니다.

[Ben Burtenshaw](https://huggingface.co/burtenshaw)와 [Maxime Labonne](https://huggingface.co/mlabonne)이 만들었습니다.

이것은 최소한의 예제입니다. 전체 예제는 [과정](https://huggingface.co/course/en/chapter12/1)의 GRPO 장을 참조하십시오.

## 종속성 설치

In [ ]:
!pip install -qqq datasets==3.2.0 transformers==4.47.1 trl==0.14.0 peft==0.14.0 accelerate==1.2.1 bitsandbytes==0.45.2 wandb==0.19.7 --progress-bar off
!pip install -qqq flash-attn --no-build-isolation --progress-bar off

## 데이터 세트 로드

In [ ]:
import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer

# Weights & Biases에 로그인
wandb.login()

# 데이터 세트 로드
dataset = load_dataset("mlabonne/smoltldr")
print(dataset)

## 모델 로드

In [ ]:
# 모델 로드
model_id = "HuggingFaceTB/SmolLM-135M-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# LoRA 로드
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

## 보상 함수 정의

In [ ]:
# 보상 함수
def reward_len(completions, **kwargs):
    return [-abs(50 - len(completion)) for completion in completions]

## 훈련 인수 정의

In [ ]:
# 훈련 인수
training_args = GRPOConfig(
    output_dir="GRPO",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    max_prompt_length=512,
    max_completion_length=96,
    num_generations=8,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=True,
    report_to=["wandb"],
    remove_unused_columns=False,
    logging_steps=1,
)

# 트레이너
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_len],
    args=training_args,
    train_dataset=dataset["train"],
)

# 모델 훈련
wandb.init(project="GRPO")
trainer.train()

## 허브에 모델 푸시

In [ ]:
# 모델 저장
merged_model = trainer.model.merge_and_unload()
merged_model.push_to_hub("<your-model-id>", private=False)

## 텍스트 생성

In [ ]:
prompt = """
# 고양이에 대한 긴 문서

고양이(Felis catus)는 집고양이 또는 반려묘라고도 하며, 작고 길들여진 육식 포유류입니다. 고양잇과에서 유일하게 길들여진 종입니다.
고고학과 유전학의 발전으로 고양이의 가축화는 기원전 7500년경 근동에서 일어났다는 것이 밝혀졌습니다.
일반적으로 애완동물과 농장 고양이로 길러지지만, 인간과의 접촉을 피하는 야생 고양이로 자유롭게 돌아다니기도 합니다.
인간에게는 동반자 관계와 해충을 죽이는 능력으로 가치가 있습니다. 수축 가능한 발톱은 쥐와 같은 작은 먹잇감을 죽이는 데 적합합니다.
강하고 유연한 몸, 빠른 반사 신경, 날카로운 이빨을 가지고 있으며, 야간 시력과 후각이 잘 발달되어 있습니다.
사회적인 종이지만, 단독 사냥꾼이자 박명박모성 포식자입니다. 고양이의 의사소통에는 야옹거리는 소리, 가르랑거리는 소리, 트릴링하는 소리, 하악질하는 소리, 으르렁거리는 소리, 끙끙거리는 소리와 같은 발성뿐만 아니라 몸짓 언어도 포함됩니다.
인간의 귀에는 너무 희미하거나 너무 높은 주파수의 소리, 예를 들어 작은 포유류가 내는 소리를 들을 수 있습니다. 페로몬을 분비하고 감지합니다.
"""

messages = [
    {"role": "user", "content": prompt},
]

In [ ]:
# 텍스트 생성
from transformers import pipeline

generator = pipeline("text-generation", model="<your-model-id>")

## 또는 이전에 정의한 모델과 토크나이저 사용
# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

generate_kwargs = {
    "max_new_tokens": 256,
    "do_sample": True,
    "temperature": 0.5,
    "min_p": 0.1,
}

generated_text = generator(messages, generate_kwargs=generate_kwargs)

print(generated_text)